In [1]:
import telebot
from telebot import types

In [2]:
quiz_database = {
0: {
    'questions':{
            0: "What is the capital of France?",
            1: "What is the largest planet in our solar system?",
            2: "What is 2 + 2?",
            # Add more questions here...
    },
    'answers':{
            0: ["Paris", "Moscow"],
            1: ["Jupiter", "Mercury"],
            2: [ "4", "5"],
            # Add more answers here...
        },
    'correct_answers': {
            0: "Paris",
            1: "Jupiter",
            2: "4"
        }
   
   },

1: {
    'questions':{
        0: '1 + 1 = ?',
        1: '3 + 3 = ?',
    },
    'answers':{
        0:['2', '3'],
        1:['7', '6'],
    },
    'correct_answers':{
        0: '2',
        1: '6'
    }
}
}


class Quiz_Get:
    def __init__(self, bot, quiz_id=None):

        self.bot = bot
        self.quiz_id = quiz_id
        
        quiz_db = quiz_database[self.quiz_id]
        self.questions = quiz_db['questions']
        self.answers = quiz_db['answers']
        self.correct_answers = quiz_db['correct_answers']

        self.current_question = 0
        self.score = 0

    def send_question(self, message):
        if self.current_question in self.questions:
            question_text = self.questions[self.current_question]
            markup = self.create_answer_markup(self.current_question)
            self.bot.send_message(message.chat.id, question_text, reply_markup=markup)
            self.bot.register_next_step_handler(message, self.check_answer)
        else:
            return self.finish_test(message)

    def create_answer_markup(self, question_num):
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        answer_options = self.answers[self.current_question] 
        for option in answer_options:
            markup.add(types.KeyboardButton(option))
        return markup


    def check_answer(self, message):
        user_answer = message.text.strip()
        correct_answer = self.correct_answers[self.current_question]

        if user_answer.lower() == correct_answer.lower():
            self.score += 1

        self.current_question += 1
        self.send_question(message)

    def finish_test(self, message):
        self.bot.send_message(message.chat.id, f"Your score: {self.score}/{len(self.questions)}")
        # You can implement further actions here, like saving scores to a database, etc.
        self.current_question = 1
        self.score = 0
        
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        markup.add(types.KeyboardButton('/start'))
        self.bot.send_message(message.chat.id, f"Начните новый тест нажав на кнопку старт",reply_markup=markup)

class Quiz_Create:
    def __init__(self, bot, quiz_id):
        self.quiz_id = quiz_id
        self.bot = bot
        self.current_quiz = {
            'questions':{},
            'answers':{},
            'correct_answers':{}
        }
        self.in_progress = True


    def add_quiz(self, message):
        if self.in_progress:
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
            markup.add(types.KeyboardButton('Добавить вопрос'))
            markup.add(types.KeyboardButton('Завершить'))
            #markup.add(types.KeyboardButton('Посмотреть тест'))
            
            self.bot.send_message(message.chat.id, "Выбери подходящий вариант" ,reply_markup=markup)
            self.bot.register_next_step_handler(message, self.handle_step)
        else:
            return self.finish_test(message)

    def handle_step(self, message):
        
        if message.text == 'Добавить вопрос':
            self.bot.send_message(message.chat.id, "Введите вопрос для теста" ,reply_markup=types.ReplyKeyboardRemove())
            self.bot.register_next_step_handler(message, self.save_question)

        elif message.text == 'Завершить':
            quiz_database[len(quiz_database)] = self.current_quiz
            
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
            markup.add(types.KeyboardButton('Создать'))
            markup.add(types.KeyboardButton('Пройти'))
            self.bot.send_message(message.chat.id, "Выбери подходящий вариант" ,reply_markup=markup)
        
    
        else :
            self.bot.send_message(message.from_user.id, 'Я пока не знаю такую команду')
            self.add_quiz(message)

    def save_question(self, message):
        q = self.current_quiz['questions']
        q[len(q)] = message.text
        self.bot.send_message(message.chat.id, "Введите варианты ответов через запятую: ",reply_markup=types.ReplyKeyboardRemove())
        self.bot.send_message(message.chat.id, "Пример: Париж, Берлин, Белград",reply_markup=types.ReplyKeyboardRemove())
        self.bot.register_next_step_handler(message, self.save_answers)

    def save_answers(self, message):
        
        q = self.current_quiz['answers']
        answs = message.text.split(',')
        answs = [i.strip() for i in answs]
        print(answs)
        if len(answs) < 2 or len(answs) > 4:
            self.bot.send_message(message.chat.id, "Нужно от 2 до 4 вариантов ответов", reply_markup=types.ReplyKeyboardRemove())
            self.bot.register_next_step_handler(message, self.save_answers)
            
        q[len(q)] = answs
        self.bot.send_message(message.chat.id, "Введите правильный ответ: ",reply_markup=types.ReplyKeyboardRemove())
        self.bot.send_message(message.chat.id, "Правильный ответ должен в точности совпадать с одним из вариантов",reply_markup=types.ReplyKeyboardRemove())
        
        self.bot.register_next_step_handler(message, self.save_correct_answer)

    def save_correct_answer(self, message):
        print(message.text)
        print(self.current_quiz['answers'])
        if message.text in self.current_quiz['answers'][0]:
        
            q = self.current_quiz['correct_answers']
            q[len(q)] = message.text
            self.bot.send_message(message.chat.id, "Вопрос сохранен",reply_markup=types.ReplyKeyboardRemove())
            self.add_quiz(message)
        else:
            self.bot.send_message(message.chat.id, "Я же сказал! Правильный ответ должен в точности совпадать с одним из вариантов",reply_markup=types.ReplyKeyboardRemove())
            self.bot.register_next_step_handler(message, self.save_correct_answer)

    def finish_saves(self, message):
        self.bot.send_message(message.chat.id, "Вопрос сохранен",reply_markup=types.ReplyKeyboardRemove())
        self.add_quiz(message)
        

In [3]:
class TestBot:
    def __init__(self, token):
        self.bot = telebot.TeleBot('6411693392:AAHRgaEm8mcS0GJmY8A2IrWSWSuXhtfE_OU')
        

    
    def get_text_messages(self, message):

        if message.text == 'Пройти':
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
            for i in quiz_database:
                markup.add(types.KeyboardButton(str(i)))
                
            self.bot.send_message(message.chat.id, "Выбери подходящий вариант" ,reply_markup=markup)

        elif message.text == 'Создать':
            self.quiz_updater = Quiz_Create(self.bot, len(quiz_database))
            self.quiz_updater.add_quiz(message)
            
            
        elif message.text.isdigit():
            quiz_id = int(message.text)
            self.quiz = Quiz_Get(self.bot, quiz_id)
            self.bot.send_message(message.from_user.id, f'Запускаю тест номер {quiz_id}')
            self.quiz.send_question(message)
    
        else :
            self.bot.send_message(message.from_user.id, 'Я пока не знаю такую команду')
            self.start_test(message)
    
    def start_test(self, message):
        self.bot.send_message(message.chat.id, "Привет! Сейчас ты можешь создать тест или пройти его")
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        markup.add(types.KeyboardButton('Создать'))
        markup.add(types.KeyboardButton('Пройти'))
        self.bot.send_message(message.chat.id, "Выбери подходящий вариант" ,reply_markup=markup)
        

# Replace 'YOUR_TELEGRAM_BOT_TOKEN' with your actual bot token.
bot_token = 'YOUR_TELEGRAM_BOT_TOKEN'
test_bot = TestBot(bot_token)

@test_bot.bot.message_handler(commands=['start'])
def handle_start(message):
    test_bot.start_test(message)

@test_bot.bot.message_handler(content_types=['text'])
def handle_message(message):
    test_bot.get_text_messages(message)
    

In [4]:
test_bot.bot.polling()

In [91]:
quiz_database

{0: {'questions': {1: 'What is the capital of France?',
   2: 'What is the largest planet in our solar system?',
   3: 'What is 2 + 2?'},
  'answers': {1: ['Paris', 'Moscow'],
   2: ['Jupiter', 'Mercury'],
   3: ['4', '5']},
  'correct_answers': {1: 'Paris', 2: 'Jupiter', 3: '4'}},
 1: {'questions': {1: '1 + 1 = ?', 2: '3 + 3 = ?'},
  'answers': {1: ['2', '3'], 2: ['7', '6']},
  'correct_answers': {1: '2', 2: '6'}}}

In [141]:
quiz_database

{0: {'questions': {0: 'What is the capital of France?',
   1: 'What is the largest planet in our solar system?',
   2: 'What is 2 + 2?'},
  'answers': {0: ['Paris', 'Moscow'],
   1: ['Jupiter', 'Mercury'],
   2: ['4', '5']},
  'correct_answers': {0: 'Paris', 1: 'Jupiter', 2: '4'}},
 1: {'questions': {0: '1 + 1 = ?', 1: '3 + 3 = ?'},
  'answers': {0: ['2', '3'], 1: ['7', '6']},
  'correct_answers': {0: '2', 1: '6'}},
 2: {'questions': {0: '2/2', 1: '3/3'},
  'answers': {0: ['1', '2', '3'], 1: ['1', '2', '3', '4']},
  'correct_answers': {0: '1', 1: '1'}}}